<a href="https://colab.research.google.com/github/arianna1994/Face-Detection/blob/main/Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Detection per Fotocamere Digitali

### Per il progetto sono stati definiti due set di dati, uno per l'addestramento e uno per il test. Per ognuno dei due set sono state predisposte due cartelle separate, una conetenete le foto con facce di persone e una contenente le foto sonza facce di persone.
### Una volta addestrato il modello di classificazione che prevede due classi ("altro" e "facce"), utilizzo l'algoritmo Haar Cascade per individuare le posizioni dei visi delle persone nelle immagini che sono state classificate con la lable "facce".

### I dataset utilizzati sono stati scaricati da Kaggle

In [ ]:
pip install img2vec_pytorch

### Importo librerie utili

In [ ]:
import os
from sklearn.pipeline import Pipeline
from img2vec_pytorch import Img2Vec
from PIL import Image
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import cv2

### Preparo il set di addestramento e il set di test

In [ ]:

img2vec = Img2Vec()

data_dir = "/content/"

train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'test')

data = {}
for j, dir_ in enumerate([train_dir, val_dir]):
    features = []
    labels = []
    for category in os.listdir(dir_):
        for img_path in os.listdir(os.path.join(dir_, category)):
            img_path_ = os.path.join(dir_, category, img_path)
            img = Image.open(img_path_)

            img_features = img2vec.get_vec(img)

            features.append(img_features)
            labels.append(category)

    data[['training_data', 'validation_data'][j]] = features
    data[['training_labels', 'validation_labels'][j]] = labels


#Definisco il modello utilizzando l'algoritmo RandomForestClassifier

In [ ]:
#creo Pipeline
my_pipe = Pipeline(
    steps=[
        ('classifier', RandomForestClassifier(random_state=0))
    ]
)

In [ ]:
# Addestro il modello per la classificazione delle immagini
my_pipe.fit(data['training_data'], data['training_labels'])

# Verifico le performance del modello
y_pred = my_pipe.predict(data['validation_data'])
score = accuracy_score(y_pred, data['validation_labels'])

#L'accuracy è molto alta
print(score)

0.9893333333333333


### Applicazione del modello

In [ ]:
#Applico il modello di classificazione su un immagine
img_path_es = "/content/test/facce/00001943.jpg"
img_es = Image.open(img_path_es)

img_es_ = img2vec.get_vec(img_es)
list_t = []
list_t.append(img_es_)

#Applico il modello di classificazione sull'immagine in analisi
y_pred = my_pipe.predict(list_t)
y_pred

array(['facce'], dtype='<U5')

In [ ]:
#Importo l'algoritmo per la face detection
face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_alt.xml')

#applico la face detection solo se la foto viene classificata come "facce" dal modello di classificazione
if y_pred[0] == "facce":
    img=cv2.imread(img_path_es)
    gray=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray=cv2.equalizeHist(gray)
    faces=face_cascade.detectMultiScale(gray, 1.25,15)
    print('Le facce sono', len(faces), '\nLe coordinate sono', '\n',faces)
else:
    print("Nessuna faccia rilevata")

Le facce sono 5 
Le coordinate sono 
 [[145  94 133 133]
 [555  97 142 142]
 [289 184 141 141]
 [870 188 174 174]
 [718 196 134 134]]
